In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets import Dataset

from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
from transformers.utils import is_torch_bf16_gpu_available
from transformers import AutoModelForCausalLM

from jigsaw.settings import DATA_DIR, MODEL_DIR

name = "no_examples"
train_path = DATA_DIR / f"ds_{name}_train.ds"
eval_path = DATA_DIR / f"ds_{name}_eval.ds"

train_dataset = Dataset.load_from_disk(str(train_path))
eval_dataset = Dataset.load_from_disk(str(eval_path))

lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    lora_dropout=0.1,
    bias="none",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    task_type="CAUSAL_LM",
)

training_args = SFTConfig(
    report_to="wandb",
    num_train_epochs=3,
    adam_epsilon=1e-6,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    optim="paged_adamw_8bit",
    learning_rate=1e-4,  # keep high, lora usually likes high.
    weight_decay=0.01,
    max_grad_norm=1.0,
    lr_scheduler_type="cosine",
    warmup_steps=40,
    bf16=is_torch_bf16_gpu_available(),
    fp16=not is_torch_bf16_gpu_available(),
    dataloader_pin_memory=True,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    save_strategy="no",
    packing=False,
    remove_unused_columns=False,
    completion_only_loss=True,
)

/home/felix/repos/jigsaw/.venv/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/felix/repos/jigsaw/.venv/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an 

In [5]:
train_dataset[0]

{'prompt': [{'content': "You are an unbiased moderator that classifies comments as violating a rule or not. You are moderating for the subreddit <subreddit> r/Futurology </subreddit>. The rule is <rule> No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed. </rule>. Does the given comment violate the rule? Respond with 'yes' or 'no'.Comment: <comment>Banks don't want you to know this! Click here to know more!</comment>",
   'role': 'user'}],
 'completion': [{'content': 'no', 'role': 'assistant'}]}

In [6]:
BASE_MODEL_PATH = "Qwen/Qwen3-0.6B"  # Model to fine-tune
LORA_PATH = MODEL_DIR / f"lora_checkpoint_{name}.pt"  # Where to save LoRA weights

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH, dtype="auto", device_map="auto"
)

In [8]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    peft_config=lora_config,
)

trainer.train()
trainer.save_model(LORA_PATH)

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
wandb: Currently logged in as: felix-newman (project-vesuvius) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.332500
20,0.129800
30,0.108300
40,0.131700
50,0.085100
60,0.083200
70,0.098900
80,0.085700
90,0.071400
100,0.065500


In [7]:
trainer.train_dataset[0]

{'prompt': [{'content': "You are an unbiased moderator that classifies comments as violating a rule or not. You are moderating for the subreddit <subreddit> r/Futurology </subreddit>. The rule is <rule> No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed. </rule>. Does the given comment violate the rule? Respond with 'yes' or 'no'.Comment: <comment>Banks don't want you to know this! Click here to know more!</comment>",
   'role': 'user'}],
 'completion': [{'content': 'no', 'role': 'assistant'}],
 'input_ids': [151644,
  872,
  198,
  2610,
  525,
  458,
  73215,
  59427,
  429,
  536,
  9606,
  6042,
  438,
  44362,
  264,
  5912,
  476,
  537,
  13,
  1446,
  525,
  13303,
  1095,
  369,
  279,
  62092,
  366,
  1966,
  36933,
  29,
  435,
  12318,
  332,
  2798,
  35681,
  690,
  1966,
  36933,
  14276,
  576,
  5912,
  374,
  366,
  12937,
  29,
  2308,
  46323,
  25,
  81667,
  11,
  44780,
  7746,
  11,
  6975,
  91104,
  12876,
 